## K-nearest neighbors (KNN) algorithm written from scratch
___

**Dataset**: Penguins


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.datasets import load_wine
import operator

In [ ]:
penguins = sns.load_dataset('penguins').dropna()

In [ ]:
# Assign colors to each species for visualization
colors = dict(zip(penguins['species'].unique(), ["red", "magenta", "lightseagreen"]))

fig, ax = plt.subplots(figsize=(9, 6))
for s, c in colors.items():
    plot_i = penguins[penguins.species == s]  # Plot each species with its corresponding color
    ax.scatter(plot_i.bill_length_mm, plot_i.bill_depth_mm, color=c, label=s)

ax.set_xlabel('Bill Length (mm)')
ax.set_ylabel('Bill Depth (mm)')
ax.legend()
plt.show()

In [ ]:
X = penguins[['bill_length_mm', 'bill_depth_mm']].to_numpy()  # Get feature and label arrays
y = penguins['species'].to_numpy()

# Split feature and label data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [ ]:
def d(a, b):
    """
    The Euclidian distance between two vectors.
    :param a: Vector 1
    :param b: Vector 2
    :return: Euclidean Distance
    """
    return np.sqrt((a-b) @ (a-b))

from em_el.utils import euclidean

In [ ]:
def k_nearest(target, X, y, K):
    """
    Finds the K nearest neighbors of a given point in a dataset.
    
    :param X: A numpy array of shape (n_samples, n_features) containing the feature vectors of the training data.
    :param y: A numpy array of shape (n_samples,) containing the corresponding labels for the training data.
    :param x_n: A numpy array of shape (n_features,) representing the query point.
    :param K: The number of nearest neighbors to return.
    
    :return: A list of tuples, where each tuple contains:
            - The distance to the query point.
            - The feature vector of the neighbor.
            - The label of the neighbor.
        The list is sorted in ascending order of distance.
    """
    distances = [(d(x, target), x, y) for x, y in zip(X, y)]
    distances = sorted(distances, key=operator.itemgetter(0))  # Sort the list by the distances
    
    return distances[:K]


In [ ]:
def KNN(target, X, y, K, regression=False):
    """
    Implements the K-Nearest Neighbors (KNN) algorithm for classification or regression.
    
    For classification:
        - Finds the K nearest neighbors to the `target` point.
        - Returns the most frequent label among the K nearest neighbors.

    For regression:
        - Finds the K nearest neighbors to the `target` point.
        - Returns the average target value of the K nearest neighbors.
    
    :param X: A numpy array containing the feature vectors of the training data.
    :param y: A numpy array containing the labels or target values of the training data.
    :param target: A numpy array representing the feature vector of the point to classify or regress.
    :param K: The number of nearest neighbors to consider (default is 3).
    :param regression: A boolean flag indicating whether to perform regression (True) or classification (False).

    :return: predicted label or target value for the input `target` point.
    """
    neighbors = k_nearest(target, X, y, K)
    if regression:
        return np.mean([x[2] for x in neighbors])
    else:
        labels = [n[2] for n in neighbors]
        return max(labels, key=labels.count)

In [ ]:
KNN(X_train[1], X_train, y_train, 4)

In [ ]:
def classification_error(train_X, train_y, test_X, test_y, K):
    """
    Calculates the classification error rate of a K-Nearest Neighbors (KNN) classifier.

    :param test_X: A numpy array containing the feature vectors of the test data.
    :param test_y: A numpy array containing the true labels of the test data.
    :param train_X: A numpy array containing the feature vectors of the training data.
    :param train_y: A numpy array containing the true labels of the training data.
    :param K: The number of nearest neighbors to consider.

    :return: The classification error rate of the K-Nearest Neighbors (KNN) classifier, which is the proportion of misclassified test points.
    """
    error = 0
    for x_i, y_i in zip(test_X, test_y):
        error += y_i != KNN(x_i, train_X, train_y, K)
    return error / len(test_X)

In [ ]:
print("Classification Error: ", classification_error(X_train, y_train, X_test, y_test, 4))

In [ ]:
possible_k = [k for k in range(3, 26, 2)]  # Test different values of K
errors = [classification_error(X_test, y_test, X_train, y_train, k) for k in possible_k]

In [ ]:
# Plot the different error values vs k values
plt.figure(figsize = (9, 6))
plt.plot(possible_k, errors, color = 'red', marker = "o")
plt.xlabel('k', fontsize = 14)
plt.ylabel('Classification Error', fontsize = 14)
plt.xticks(possible_k)
plt.show()